In [1]:
# Import Libraries
import tweepy as tw
import pandas as pd
import numpy as np
import datetime
import pytz
from dateutil.relativedelta import relativedelta
import time

# Authentication

In [2]:
keys = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\Input\TW API Keys\API_3\apikeys_3.csv")

my_api_key = keys["Keys"][0]
my_api_secret = keys["Keys"][1]

my_access_token = keys["Keys"][2]
my_access_token_secret = keys["Keys"][3]

In [3]:
# your Twitter API key ans access token
api_key = my_api_key
api_secret = my_api_secret

access_token = my_access_token
access_token_secret = my_access_token_secret

In [4]:
# authenticate
auth = tw.OAuthHandler(my_api_key, my_api_secret)
auth.set_access_token(access_token, access_token_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit = True)

## Variables

sc_name = "el33th4xor"

In [6]:
user_df = pd.read_csv(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\Input/Top 20 Cryptocurrencies and Related TW Accounts.csv", encoding= 'unicode_escape')
user_df

,Coin,ID,Owner/Influence,Name,Screen_name,Status,Why did I choose this person?
0,Ethereum,ethereum,Vitalik Buterin,vitalik.eth,VitalikButerin,co-founder,he is also known in crypto world
1,BNB,binancecoin,Changpeng Zhao,CZ ??Binance,cz_binance,CEO,NaN
2,XRP,ripple,Brad Garlinghouse,Brad Garlinghouse,bgarlinghouse,CEO,NaN
3,ADA(Cardano),cardano,Charles Hoskinson,Charles Hoskinson,IOHK_Charles,CEO/co-founder,NaN
4,DOGE,dogecoin,Elon Musk_doge,Elon Musk,elonmusk,influencer,"as a popular figure, highly interested in this..."
5,MATIC(Polygon),matic-network,Sandeep Nailwal,Sandeep | Polygon ????3??,sandeepnailwal,co-founder,he seems more active than other co-founders
6,DOT(Polkadot),polkadot,WEB3 Foundation,Web3 Foundation,Web3foundation,foundation,there is no account of CEO or sth
7,HEX,hex,Richard Heart,Richard Heart,RichardHeartWin,CEO,NaN
8,SHIB,shiba-inu,Elon Musk_shiba,Elon Musk,elonmusk,influencer,"as a popular figure, highly interested in this..."
9,SOL,solana,Solana,Solana,solana,company,there is no account of CEO or sth


In [7]:
#datetime format -> (year, month, day, hour, minute, second)
endDate = datetime.datetime.now()
startDate = endDate - relativedelta(months = 1)
#endDate = datetime.datetime.now()
#startDate = endDate - relativedelta(days = 1) #days, months, years can be used to calculate

utc=pytz.UTC

startDate = utc.localize(startDate) 
endDate = utc.localize(endDate)

In [8]:
limit = 200

# Getting Liked Tweets

In [9]:
def get_liked_tweets(sc_name):
    
    liked_tweets = []
    get_liked_tweets = api.get_favorites(screen_name = sc_name)
    for tweet in get_liked_tweets:
        if tweet.created_at < endDate and tweet.created_at > startDate:
            liked_tweets.append(tweet)


    while (get_liked_tweets[-1].created_at > startDate):
        get_liked_tweets = api.get_favorites(screen_name = sc_name, max_id = get_liked_tweets[-1].id)
        for tweet in get_liked_tweets:
            if tweet.created_at < endDate and tweet.created_at > startDate:
                liked_tweets.append(tweet)
    return liked_tweets

In [10]:
def get_liked_tweets_2(sc_name):
    #variables
    start = time.time()
    tweets = []
    num_of_tweets = 0
    
    #initial request
    while True:
        try:
            tmpTweets = api.get_favorites(screen_name = sc_name)
            while(len(tmpTweets) == 0):
                print("empty list error_1, please try again")
                tmpTweets = api.get_favorites(screen_name = sc_name)
            for tweet in tmpTweets:
                if tweet.created_at < endDate and tweet.created_at > startDate:
                    tweets.append(tweet)
                    num_of_tweets += 1
            break
        except tw.errors.TweepyException as e:
            print("Going to sleep")
            print(e)
            time.sleep(60*15)
            continue
    
    #collection
    while True:
        try:
            while (tmpTweets[-1].created_at > startDate) and (len(tmpTweets) != 1):
                old_ID = tmpTweets[-1].id
                tmpTweets = api.get_favorites(screen_name = sc_name, max_id = old_ID)
                while(len(tmpTweets) == 0):
                    print("empty list error_2, please try again")
                    print(old_ID)
                    time.sleep(15)
                    tmpTweets = api.get_favorites(screen_name = sc_name, max_id = old_ID)
                    #print("1: ", {tmpTweets[0].created_at})
                for tweet in tmpTweets:
                    if tweet.created_at < endDate and tweet.created_at > startDate:
                        tweets.append(tweet)
                        num_of_tweets += 1
                if(len(tmpTweets) == 1):
                    print("it's the end of the limit which is: ", len(tweets))
            break
        except tw.errors.TweepyException as e:
            print(e)
            print("Going to sleep")
            time.sleep(60*15)
            continue
    
    #return        
    end = time.time()
    print(sc_name)
    print(end-start)
    return tweets

In [11]:
columns = ["Tweet_id", "Name", "Screen_name", "Tweet", "Date", "Category", "N_of_RT", "N_of_fav"]
columns_2 = ["Tweet_id", "Name", "Screen_name", "Tweet", "Date", "Category", "N_of_RT", "N_of_fav", "Related_account"]

In [12]:
def categorize_liked_tweets(liked_tweets):

    data = []
    term_rt = "RT"
    for tweet in liked_tweets:

        #for consecutive tweets, next one is shown primaryly in data frame, I can check with using hours and minutes if necessary
        date = tweet.created_at
        #date = date.strftime('%Y-%m-%d')

        #retweet and favorite counts
        number_of_retweet = tweet.retweet_count
        number_of_favorite = tweet.favorite_count


        #it's gonna be direct tweet if all the checks below fail
        category_specified = "direct"

        #retweet check
        if term_rt in tweet.text.split():

            category_specified = "retweet"

        #quote(rt with comment) check
        elif tweet.is_quote_status is True:
            category_specified = "quote"

        #reply check
        elif tweet.in_reply_to_status_id is not None:
            category_specified = "reply"

        data.append([tweet.id, tweet.user.name, tweet.user.screen_name, tweet.text, date, category_specified, number_of_retweet, number_of_favorite])
    
    df_met = pd.DataFrame(data, columns = columns)
    return df_met 

In [13]:
def LikedTweetCollector(user_df):
    df = pd.DataFrame(columns = columns_2)
    index_counter_1 = 0;
    
    for x in user_df.Screen_name:
    
        taken_tweets = get_liked_tweets_2(x)
        df_met_2 = categorize_liked_tweets(taken_tweets)

        df_met_2["Related_account"] = user_df["Screen_name"][index_counter_1]
        index_counter_1 += 1
        
        df = df.append(df_met_2)
    
    return df

In [14]:
#manuel collection of liked tweets
"""df = pd.DataFrame(columns = columns)
for x in user_df.Screen_name:
    
    taken_liked_tweets = get_liked_tweets_2(x)
    df_met_2 = categorize_liked_tweets(taken_liked_tweets)
    df_met_2["Related_account"] = sc_name
    df = df.append(df_met_2)"""

'df = pd.DataFrame(columns = columns)\nfor x in user_df.Screen_name:\n    \n    taken_liked_tweets = get_liked_tweets_2(x)\n    df_met_2 = categorize_liked_tweets(taken_liked_tweets)\n    df_met_2["Related_account"] = sc_name\n    df = df.append(df_met_2)'

In [ ]:
start = time.time()
df = LikedTweetCollector(user_df)
end = time.time()
print(end - start)

VitalikButerin
0.46295785903930664


Rate limit reached. Sleeping for: 501


In [ ]:
df.drop_duplicates(inplace = True)
df.reset_index(inplace = True, drop = True)

In [ ]:
df.shape

In [ ]:
#null check --> if it is "False", we all ok
df.isna().any().any()

In [ ]:
df.info()

In [ ]:
#duplication check
df[df.duplicated(keep=False)].shape[0]

In [ ]:
df['Date'] = df['Date'].dt.tz_localize(None)

In [ ]:
pd.set_option("max_rows", None)
df.to_excel(r"C:\Users\Kutay\Desktop\My Docs\My Docs v2.0\Data Science\RA\Cryptocurrency\RA_Cryptocurrency\Data\liked_tweets.xlsx")
df.head()